This notebook uses Viola to render the notebook as a dashboard. It's hosted on the cloud for free using mybinder.org. 

### NBA Player Stats and Salary

For this article, we will be using two separate a dataset from kaggle, The salary data that can be found <a href="https://www.kaggle.com/koki25ando/salary"> here </a> and the player stats can be found <a href="https://www.kaggle.com/mcamli/nba17-18#nba.csv"> here. </a> Both dataset are from the season 2017 - 2018.

In [1]:
%%capture
!pip install -U altair vega_datasets notebook vega

In [2]:
%%capture
import numpy as np 
import pandas as pd 
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
from ipywidgets import interact
import altair as alt

%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',False)
pd.set_option('display.max_rows',100)
sns.set(style='ticks')
alt.renderers.enable('kaggle')

In [3]:
salary = pd.read_csv('NBA_season1718_salary.csv')
stats = pd.read_csv('nba_extra.csv')

First, we will transform the ```Player``` column in the stats data, so we can match in with the salary data. We'll also drop the first column in the salary data.



##### The first few rows of the Salary Data:


In [4]:
salary.head()

,Unnamed: 0,Player,Tm,season17_18
0,1,Stephen Curry,GSW,34682550.0
1,2,LeBron James,CLE,33285709.0
2,3,Paul Millsap,DEN,31269231.0
3,4,Gordon Hayward,BOS,29727900.0
4,5,Blake Griffin,DET,29512900.0


##### The first few rows of the Stats Data:

In [5]:
salary.drop('Unnamed: 0',axis=1,inplace=True)
salary.columns = ['Player','Tm','salary']
stats.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Alex Abrines\abrinal01,SG,24,OKC,75,8,1134,115,291,0.395,84,221,0.380,31,70,0.443,0.540,39,46,0.848,26,88,114,28,38,8,25,124,353
1,2,Quincy Acy\acyqu01,PF,27,BRK,70,8,1359,130,365,0.356,102,292,0.349,28,73,0.384,0.496,49,60,0.817,40,217,257,57,33,29,60,149,411
2,3,Steven Adams\adamsst01,C,24,OKC,76,76,2487,448,712,0.629,0,2,0.000,448,710,0.631,0.629,160,286,0.559,384,301,685,88,92,78,128,215,1056
3,4,Bam Adebayo\adebaba01,C,20,MIA,69,19,1368,174,340,0.512,0,7,0.000,174,333,0.523,0.512,129,179,0.721,118,263,381,101,32,41,66,138,477
4,5,Arron Afflalo\afflaar01,SG,32,ORL,53,3,682,65,162,0.401,27,70,0.386,38,92,0.413,0.485,22,26,0.846,4,62,66,30,4,9,21,56,179


In [6]:
new=[]
for i in range(0,len(stats.Player)):
    x=stats.Player[i].split("\\")
    new.append(x[0])
stats["Player"]=new

In [7]:
data = pd.merge(salary,stats,on=['Player','Tm'])

- Note that this dataset contains duplicate rows, because players can switch teams in a single season. We've also renamed the ```salary``` column.
- Also notice that for the missing values, this can be easily obtained (i.e. FG% = FG/GA) except for eFG%.
- For the missing values in eFG%, we'll replace them by the average of their respective ```Position```.
- For duplicate rows, we'll keep the most recent team.

#### Handling Missing Values

In [8]:
missing = pd.DataFrame(data.isnull().sum())
missing.columns = ['% of missing']
missing['% of missing'] = missing['% of missing']/data.shape[0]
missing[missing['% of missing']>0]

,% of missing
FG%,0.004107
3P%,0.090349
2P%,0.024641
eFG%,0.004107
FT%,0.073922


In [9]:
data.rename(columns={'eFG%': 'eFGRate',}, inplace=True)
def show_average_by_pos(metric):
    guide = pd.pivot_table(index='Pos',values=metric,aggfunc=np.mean,data=data[['Pos',metric]])
    return guide

show_average_by_pos('eFGRate')

,eFGRate
Pos,
C,0.544529
PF,0.511559
PG,0.475516
SF,0.479988
SG,0.505589


In [10]:
def imputer(cols):
    eFGRate = cols[0]
    Pos = cols[1]
    
    if pd.isnull(eFGRate):

        if Pos == 'C':
            return 0.544529
        elif Pos == 'PF':
            return 0.511559
        elif Pos == 'PG':
            return 0.475516
        elif Pos == 'SF':
            return 0.479988
        else:
            return 0.505589
    else:
        return eFGRate

- Uppon further investigation, it seems that the missing values are caused by 0 ``FGA``, 0 ```3PA``` and so on. It's best to impute them by 0.

In [11]:
data['eFGRate'] = data[['eFGRate','Pos']].apply(imputer,axis=1)
data['FG%'] = data['FG'] / data['FGA']
data['3P%'] = data['3P'] / data['3PA']
data['2P%'] = data['2P'] / data['2PA']
data['FT%'] = data['FT'] / data['FTA']
missing = pd.DataFrame(data.isnull().sum())
missing.columns = ['% of missing']
missing['% of missing'] = missing['% of missing']/data.shape[0]
missing[missing['% of missing']>0]

,% of missing
FG%,0.004107
3P%,0.090349
2P%,0.024641
FT%,0.073922


In [12]:
data.fillna(0,inplace=True)
missing = pd.DataFrame(data.isnull().sum())
missing.columns = ['% of missing']
missing['% of missing'] = missing['% of missing']/data.shape[0]
missing[missing['% of missing']>0]

,% of missing


#### Handling Duplicate ```Player```

In [13]:
ids = data["Player"]
print('There are',data[ids.isin(ids[ids.duplicated()])].sort_values(by="Player").shape[0],'duplicate players.')

There are 36 duplicate players.


#### Duplicate Players

In [14]:
data[ids.isin(ids[ids.duplicated()])].sort_values(by="Player")

,Player,Tm,salary,Rk,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFGRate,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
478,Antonius Cleveland,DAL,50000.0,100,SG,23,13,0,81,4,14,0.285714,0,1,0.000000,4,13,0.307692,0.286,2,4,0.500000,2,8,10,2,6,4,2,11,10
454,Antonius Cleveland,ATL,92160.0,100,SG,23,4,0,42,4,7,0.571429,3,3,1.000000,1,4,0.250000,0.786,2,2,1.000000,1,3,4,0,1,1,5,12,13
460,Briante Weber,MEM,83129.0,506,PG,25,5,0,119,10,21,0.476190,0,2,0.000000,10,19,0.526316,0.476,4,6,0.666667,5,12,17,9,8,2,8,10,24
475,Briante Weber,HOU,50000.0,506,PG,25,13,0,118,9,22,0.409091,4,9,0.444444,5,13,0.384615,0.500,4,4,1.000000,3,15,18,13,10,3,7,18,26
441,DeAndre Liggins,NOP,172238.0,297,SG,29,27,3,244,18,41,0.439024,8,17,0.470588,10,24,0.416667,0.537,0,0,0.000000,2,24,26,21,10,4,7,20,44
331,DeAndre Liggins,MIL,1577230.0,297,SG,29,31,1,480,22,65,0.338462,11,36,0.305556,11,29,0.379310,0.423,2,5,0.400000,5,44,49,27,27,10,13,61,57
462,Demetrius Jackson,HOU,50000.0,244,PG,23,12,0,63,4,14,0.285714,0,6,0.000000,4,8,0.500000,0.286,0,0,0.000000,4,7,11,5,4,1,9,6,8
461,Demetrius Jackson,HOU,74159.0,244,PG,23,12,0,63,4,14,0.285714,0,6,0.000000,4,8,0.500000,0.286,0,0,0.000000,4,7,11,5,4,1,9,6,8
171,Ersan Ilyasova,ATL,6000000.0,235,PF,30,46,40,1175,184,401,0.458853,61,170,0.358824,123,231,0.532468,0.535,72,90,0.800000,74,177,251,51,44,17,43,92,501
436,Ersan Ilyasova,PHI,502328.0,235,PF,30,23,3,554,93,212,0.438679,30,83,0.361446,63,129,0.488372,0.509,33,45,0.733333,44,110,154,39,17,9,29,65,249


In [15]:
data.drop_duplicates(subset ="Player", keep = 'first', inplace = True) 

## Data Visualization

### Interactive Cross Filter of Player Salary, FG% and FT%.

- Brushing on one histogram, affects the other histogram.

In [16]:
datat = data.copy()
brush = alt.selection(type='interval', encodings=['x'])

# Define the base chart, with the common parts of the
# background and highlights
base = alt.Chart().mark_bar().encode(
    x=alt.X(alt.repeat('column'), type='quantitative', bin=alt.Bin(maxbins=30)),
    y='count()'
).properties(
    width=250,
    height=250
)

# blue background with selection
background = base.add_selection(brush)

# yellow highlights on the transformed data
highlight = base.encode(
    color=alt.value('goldenrod')
).transform_filter(brush)

# layer the two charts & repeat
alt.layer(
    background,
    highlight,
    data=datat
).repeat(column=["salary", "FG%", "FT%"])

alt.RepeatChart(...)

### Cross filter of any columns

In [17]:
def cross_filter(column1='FG%',column2='Age',column3='3P'):
    brush = alt.selection(type='interval', encodings=['x'])

    # Define the base chart, with the common parts of the
    # background and highlights
    base = alt.Chart().mark_bar().encode(
        x=alt.X(alt.repeat('column'), type='quantitative', bin=alt.Bin(maxbins=30)),
        y='count()'
    ).properties(
        width=250,
        height=250
    )

    # blue background with selection
    background = base.add_selection(brush)

    # yellow highlights on the transformed data
    highlight = base.encode(
        color=alt.value('goldenrod')
    ).transform_filter(brush)

    # layer the two charts & repeat
    plot = alt.layer(
        background,
        highlight,
        data=datat
    ).repeat(column=[column1,column2,column3])
    return plot

In [18]:
interactive = interact(cross_filter,column1=list(data.columns[2:]),column2=list(data.columns[2:]),column3=list(data.columns[2:]))

interactive(children=(Dropdown(description='column1', index=9, options=('salary', 'Rk', 'Pos', 'Age', 'G', 'GS…

In [19]:
def scatter_plot(metric='PTS'):
    scatter = alt.Chart(datat).mark_circle().encode(
        x = metric,
        y = 'salary',
        color = 'Pos',
        tooltip = 'Player'
    )
    
    return scatter

#### This plot is an Interactive Scatter Plot with Player Name Tooltip.

In [20]:
interactive_scatter = interact(scatter_plot,metric=list(data.columns[5:]))

interactive(children=(Dropdown(description='metric', index=25, options=('Age', 'G', 'GS', 'MP', 'FG', 'FGA', '…

### Average Salary Per Team

#### Cleveland has the highest Average Salary.

In [21]:
alt.Chart(data).mark_bar().encode(
    x = 'Tm',
    y = 'average(salary)',
    tooltip = ['average(salary)'],
    color=alt.condition(
    alt.datum.Tm == 'CLE',  # If the year is 1810 this test returns True,
    alt.value('orange'),     # which sets the bar orange.
    alt.value('steelblue')   # And if it's not true it sets the bar steelblue.
)
)



alt.Chart(...)

#### Stephen Curry Makes More Money than LeBron James, despite making significantly smaller Points.

In [22]:
bar = alt.Chart(data[(data['Player']=='LeBron James')|(data['Player']=='Stephen Curry')]).mark_bar().encode(
    y = 'Player:N',
    x = 'salary',

)
bar2 = alt.Chart(data[(data['Player']=='LeBron James')|(data['Player']=='Stephen Curry')]).mark_bar().encode(
    y = 'Player:N',
    x = 'PTS',

)

bar&bar2

alt.VConcatChart(...)

### Machine Learning

- We'll consider the columns that are rates as features, and we'll keep all other metrics.
- We'll also drop ```Player``` and ```Tm``` so that our model only pick up patterns from metrics themselves.
- We'll do a one hot encodinig on the ```Pos``` column.

In [23]:
data = data.drop(['Player','Tm','FG','FGA','3P','3PA','2P','2PA','FT','FTA','Pos'],axis=1)
data.head(10)

,salary,Rk,Age,G,GS,MP,FG%,3P%,2P%,eFGRate,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,34682550.0,120,29,51,51,1631,0.495370,0.423154,0.595041,0.618,0.920530,36,225,261,310,80,8,153,114,1346
1,33285709.0,248,33,82,82,3026,0.542405,0.366995,0.603066,0.590,0.730697,97,612,709,747,116,71,347,136,2251
2,31269231.0,341,32,38,37,1143,0.464368,0.345133,0.506211,0.509,0.695652,65,180,245,105,39,44,73,99,555
3,29727900.0,207,27,1,1,5,0.500000,0.000000,1.000000,0.500,0.000000,0,1,1,0,0,0,0,1,2
4,29512900.0,191,28,25,25,831,0.433014,0.348148,0.473498,0.489,0.783784,27,139,166,155,11,9,66,61,496
5,28703704.0,305,31,78,78,2510,0.426907,0.399329,0.474138,0.553,0.854406,66,368,434,537,85,19,183,192,1267
6,28530608.0,508,29,80,80,2914,0.448726,0.297546,0.484938,0.477,0.736749,152,652,804,820,147,20,381,200,2028
7,28530608.0,106,30,12,12,373,0.380952,0.311688,0.439560,0.452,0.803030,0,27,27,49,12,3,18,24,205
8,28299399.0,194,28,72,72,2551,0.449275,0.367036,0.530949,0.541,0.858322,41,348,389,630,126,50,315,169,2191
9,27739975.0,130,28,80,80,2711,0.456476,0.310105,0.493783,0.488,0.824687,59,256,315,417,85,22,175,151,1840


As a base model, you can never go wrong with a linear model. We'll do a train-test split, then 3- Fold cross validate on the training set.

In [24]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import metrics
X = data.drop('salary',axis=1)
X = pd.get_dummies(X)
y = data[['salary']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [25]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(normalize=True)
model.fit(X_train,y_train)
cv = np.mean(cross_val_score(model, X_train, y_train, cv=3,scoring='neg_mean_squared_error'))
print ("Linear Regression RMSE with 3 cross validation :",np.sqrt(-cv))
y_predict_test = model.predict(X_test)
score_test = np.sqrt(metrics.mean_squared_error(y_test, y_predict_test))
print('Linear Regression Test RMSE',score_test)
print('Average Salary in the Dataset',data['salary'].mean())

Linear Regression RMSE with 3 cross validation : 5592997.5461236555
Linear Regression Test RMSE 5194095.922658304
Average Salary in the Dataset 6696803.098290598


In [26]:
from sklearn.ensemble import RandomForestRegressor

Below is the performance of Random Forest. This is  achieved by running a GridSearch. Stay tuned. We'll update this notebook soon, and will employ better feature engineering techniques!

In [27]:
model = RandomForestRegressor(max_depth=9,max_features='auto',n_estimators=500)
model.fit(X_train,y_train)
cv = np.mean(cross_val_score(model, X_train, y_train, cv=3,scoring='neg_mean_squared_error'))
print ("Model RMSE with 3 cross validation :",np.sqrt(-cv))
y_predict_test = model.predict(X_test)
score_test = np.sqrt(metrics.mean_squared_error(y_test, y_predict_test))
print('Test RMSE',score_test)
print('Average Salary in the Dataset',data['salary'].mean())

Model RMSE with 3 cross validation : 4941272.855393014
Test RMSE 5593990.843845949
Average Salary in the Dataset 6696803.098290598
